## Wzorce 2d

Biblioteki:

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from time import perf_counter

1. Zaimplementuj algorytm wyszukiwania wzorca 2-wymiarowego



In [2]:
def get_diff_columns(pattern):
    columns = []
    indexes = []
    alphabet = set()
    for i in range(len(pattern[0])):
        col = []
        for j in range(len(pattern)):
            col.append(pattern[j][i])
            alphabet.add(pattern[j][i])

        if col in columns:
            idx = columns.index(col)
            indexes.append(idx)
        else:
            columns.append(col)
            indexes.append(len(columns) - 1)
    return columns, indexes, alphabet

In [3]:
def vertical_automaton(columns, letters):
    tt = [{}]
    words = [[]]
    states = [0] * len(columns)

    for i in range(len(columns[0])):
        for j in range(len(columns)):
            if columns[j][i] in tt[states[j]]:
                states[j] = tt[states[j]][columns[j][i]]
            else:
                tt[states[j]][columns[j][i]] = len(tt)
                words.append(words[states[j]] + [columns[j][i]])
                states[j] = len(tt)
                tt.append({})

    for i in range(len(tt)):
        for l in letters:
            if l not in tt[i]:
                suffix = (words[i] + [l])[1:]
                state = 0
                for s in suffix:
                    if s in tt[state]:
                        state = tt[state][s]
                    else:
                        state = 0

                tt[i][l] = state
    return tt, states

In [4]:
def horizontal_automaton(pattern, letters):
    result = []
    for state in range(len(pattern) + 1):
        result.append({})
        for l in letters:
            next_state = min(len(pattern), state + 1)
            while True:
                if pattern[:next_state] == (pattern[:state] + [l])[state - next_state + 1:state + 1]:
                    break
                next_state -= 1
            result[state][l] = next_state
    return result

In [5]:
def main_automaton(pattern):
    columns, indexes, letters = get_diff_columns(pattern)
    vertical_tt, vertical_states = vertical_automaton(columns, letters)

    new_pattern = [vertical_states[indexes[i]] for i in range(len(indexes))]
    horizontal_tt = horizontal_automaton(new_pattern, vertical_states)
    horizontal_state = len(horizontal_tt) - 1
    return vertical_tt, horizontal_tt, horizontal_state

In [6]:
def pattern_matching_2d(text, pattern, automaton=None):
    if automaton is None:
        vertical_tt, horizontal_tt, horizontal_state = main_automaton(pattern)
    else:
        vertical_tt, horizontal_tt, horizontal_state = automaton

    result = []
    vertical_states = []
    for i in range(len(text)):
        if len(text[i]) < len(vertical_states):
            vertical_states = vertical_states[:len(text[i])]
        elif len(vertical_states) < len(text[i]):
            vertical_states = vertical_states + [0] * (len(text[i]) - len(vertical_states))

        new_horizontal_state = 0
        for j in range(len(text[i])):
            if text[i][j] in vertical_tt[vertical_states[j]]:
                vertical_states[j] = vertical_tt[vertical_states[j]][text[i][j]]
            else:
                vertical_states[j] = 0
            if vertical_states[j] in horizontal_tt[new_horizontal_state]:
                new_horizontal_state = horizontal_tt[new_horizontal_state][vertical_states[j]]
                if new_horizontal_state == horizontal_state:
                    result.append((i - len(pattern) + 1, j - len(pattern[0]) + 1))
            else:
                new_horizontal_state = 0
    return result

2. Znajdź w załączonym pliku "haystack.txt" wszyskie sytuacje, gdy taka sama litera występuje na tej samej pozycji w dwóch kolejnych linijkach. Zwróć uwagę, na nierówną długość linii w pliku.


In [7]:
with open("haystack.txt") as f:
    text = f.readlines()

In [8]:
for i in range(ord("a"), ord("z") + 1):
    pattern = [chr(i), chr(i)]
    result = pattern_matching_2d(text, pattern)
    print(f"Given pattern: {pattern}")
    print(f"All indexes:\n{result}")
    print("----------------------------")

Given pattern: ['a', 'a']
All indexes:
[(0, 82), (3, 30), (5, 60), (6, 63), (20, 6), (28, 69), (31, 50), (31, 73), (33, 66), (37, 4), (52, 12), (53, 12), (53, 48), (56, 11), (57, 36), (58, 36), (59, 24), (64, 2), (64, 14), (64, 22), (65, 35), (69, 35), (76, 21), (76, 74), (77, 42), (77, 61), (78, 59), (79, 37)]
----------------------------
Given pattern: ['b', 'b']
All indexes:
[]
----------------------------
Given pattern: ['c', 'c']
All indexes:
[(3, 54), (10, 45), (13, 10), (41, 0), (68, 0), (82, 41)]
----------------------------
Given pattern: ['d', 'd']
All indexes:
[(37, 19)]
----------------------------
Given pattern: ['e', 'e']
All indexes:
[(0, 63), (1, 8), (4, 77), (7, 65), (10, 1), (10, 64), (14, 2), (15, 43), (17, 6), (18, 27), (20, 10), (21, 61), (22, 53), (24, 3), (24, 65), (28, 67), (28, 73), (29, 38), (29, 43), (37, 48), (40, 11), (40, 26), (41, 57), (42, 36), (42, 48), (46, 52), (47, 50), (51, 31), (57, 54), (58, 50), (58, 54), (59, 73), (63, 66), (65, 69), (66, 72), (

3. Znajdź wszystkie wystąpienia "th" oraz "t h" w dwóch kolejnych liniach na tej samej pozycji.



In [10]:
pattern_matching_2d(text, ["th", "th"])

[]

In [11]:
pattern_matching_2d(text, ["t h", "t h"])

[(37, 0)]

4. Wybierz przynajmniej 4 litery (małe). Znajdź wszystkie wystąpienia tej litery w załączonym pliku "haystack.png"


In [12]:
def convert_image(file_name):
    image = Image.open(file_name)
    pixels = list(image.getdata())
    width, height = image.size
    text = []
    i = width
    for pixel in pixels:
        if i == width:
            i = 0
            text.append([])
        text[-1].append(pixel[0])
        i += 1
    return text

In [13]:
text = convert_image("haystack.png")

Litera: c


In [15]:
c = convert_image("patterns/c.png")
c


[[255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255],
 [255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255],
 [255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255],
 [255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255],
 [255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255,
  255],
 [255,
  255,
  255,
  255,
  255,
  255,
  248,
  248,
  135,
  135,
  135,
  45,
  